In [1]:
import pandas as pd 
import numpy as np
import plotly.express as px
import dstack
from dstack import push_frame

## Let us import data downloaded from kaggle

In [2]:
df = pd.read_csv('covid_19_data.csv')
df

,SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
0,1,01/22/2020,Anhui,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
1,2,01/22/2020,Beijing,Mainland China,1/22/2020 17:00,14.0,0.0,0.0
2,3,01/22/2020,Chongqing,Mainland China,1/22/2020 17:00,6.0,0.0,0.0
3,4,01/22/2020,Fujian,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
4,5,01/22/2020,Gansu,Mainland China,1/22/2020 17:00,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
18641,18642,04/25/2020,Wyoming,US,2020-04-26 02:31:18,491.0,7.0,0.0
18642,18643,04/25/2020,Xinjiang,Mainland China,2020-04-26 02:31:18,76.0,3.0,73.0
18643,18644,04/25/2020,Yukon,Canada,2020-04-26 02:31:18,11.0,0.0,0.0
18644,18645,04/25/2020,Yunnan,Mainland China,2020-04-26 02:31:18,185.0,2.0,181.0


## Let's get the file containing the iso3 country codes needed for geographic visualisation

In [3]:
codes = pd.read_csv('country_codes.csv')
codes

,Unnamed: 0,Country,Code
0,0,Afghanistan,AFG
1,1,Albania,ALB
2,2,Algeria,DZA
3,3,American Samoa,ASM
4,4,Andorra,AND
...,...,...,...
217,217,Virgin Islands,VGB
218,218,West Bank,WBG
219,219,Yemen,YEM
220,220,Zambia,ZMB


## The two datasets need a few changes

In [4]:
df['ObservationDate'] = pd.to_datetime(df['ObservationDate'])
df['Last Update'] = pd.to_datetime(df['Last Update'])

df.rename(columns={
    'ObservationDate': 'Date',
    'Country/Region': 'Country'
}, inplace=True)

#We need to also rename some variables for consistency purposes, since we intend to merge two datasets
update_countries = {'Mainland China': 'China', 'Ivory Coast': 'Cote d\'Ivoire', 'The Bahamas': 'Bahamas, The', 'Bahamas': 'Bahamas, The', 
                   'The Gambia': 'Gambia, The', 'Cape Verde': 'Cabo Verde', 'US': 'United States', 'UK': 'United Kingdom',
                   'South Korea': 'Korea, South', 'North Korea': 'Korea, North', 'Republic of Ireland': 'Ireland', 
                   'St. Martin': 'Saint Martin', '(\'St. Martin\')': 'Saint Martin', 'Palestine': 'West Bank and Gaza', 
                   'occupied Palestinian territory': 'West Bank and Gaza'}

df = df.replace({'Country': update_countries})
df


,SNo,Date,Province/State,Country,Last Update,Confirmed,Deaths,Recovered
0,1,2020-01-22,Anhui,China,2020-01-22 17:00:00,1.0,0.0,0.0
1,2,2020-01-22,Beijing,China,2020-01-22 17:00:00,14.0,0.0,0.0
2,3,2020-01-22,Chongqing,China,2020-01-22 17:00:00,6.0,0.0,0.0
3,4,2020-01-22,Fujian,China,2020-01-22 17:00:00,1.0,0.0,0.0
4,5,2020-01-22,Gansu,China,2020-01-22 17:00:00,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
18641,18642,2020-04-25,Wyoming,United States,2020-04-26 02:31:18,491.0,7.0,0.0
18642,18643,2020-04-25,Xinjiang,China,2020-04-26 02:31:18,76.0,3.0,73.0
18643,18644,2020-04-25,Yukon,Canada,2020-04-26 02:31:18,11.0,0.0,0.0
18644,18645,2020-04-25,Yunnan,China,2020-04-26 02:31:18,185.0,2.0,181.0


In [5]:
update_codes = {'Congo, Democratic Republic of the': 'Congo (Kinshasa)', 'Congo, Republic of the': 'Congo (Brazzaville)',
               'Macedonia': 'North Macedonia', 'West Bank': 'West Bank and Gaza', 'Swaziland': 'Eswatini'}

codes = codes.replace({'Country': update_codes})
codes

,Unnamed: 0,Country,Code
0,0,Afghanistan,AFG
1,1,Albania,ALB
2,2,Algeria,DZA
3,3,American Samoa,ASM
4,4,Andorra,AND
...,...,...,...
217,217,Virgin Islands,VGB
218,218,West Bank and Gaza,WBG
219,219,Yemen,YEM
220,220,Zambia,ZMB


## Let's single out african countries

In [8]:
africa = ['Algeria', 'Angola', 'Benin', 'Botswana', 'Burkina Faso', 'Cabo Verde', 'Cameroon', 'Central African Republic'
         'Chad', 'Congo (Brazzaville)', 'Congo (Kinshasa)', 'Cote d\'Ivoire', 'Djibouti', 'Egypt', 
         'Equatorial Guinea', 'Eritrea', 'Eswatini', 'Ethiopia', 'Gabon', 'Gambia', 'Ghana', 'Guinea'
         'Kenya', 'Liberia', 'Madagascar', 'Mauritania', 'Mauritius', 'Morocco', 'Namibia', 'Niger',
         'Nigeria', 'Rwanda', 'Senegal', 'Seychelles', 'Sierra Leone', 'Somalia', 'South Africa', 'South Sudan', 'Sudan', 'Tanzania', 'Togo',
         'Tunisia', 'Uganda', 'Zambia', 'Zimbabwe', 'Mozambique', 'Libya', 'Guinea-Bissau', 'Mali', 'Western Sahara',
         'Sao Tome and Principe']

africa_cov = list()
for i in africa:
    df1 = df[df['Country'] == i]
    africa_cov.append(pd.DataFrame(df1))
    
africa_covid = pd.concat(i for i in africa_cov)
africa_covid

,SNo,Date,Province/State,Country,Last Update,Confirmed,Deaths,Recovered
2352,2353,2020-02-25,NaN,Algeria,2020-02-25 23:43:03,1.0,0.0,0.0
2453,2454,2020-02-26,NaN,Algeria,2020-02-25 23:43:03,1.0,0.0,0.0
2557,2558,2020-02-27,NaN,Algeria,2020-02-25 23:43:03,1.0,0.0,0.0
2665,2666,2020-02-28,NaN,Algeria,2020-02-25 23:43:03,1.0,0.0,0.0
2783,2784,2020-02-29,NaN,Algeria,2020-02-25 23:43:03,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...
17189,17190,2020-04-21,NaN,Sao Tome and Principe,2020-04-21 23:39:09,4.0,0.0,0.0
17509,17510,2020-04-22,NaN,Sao Tome and Principe,2020-04-22 23:38:57,4.0,0.0,0.0
17828,17829,2020-04-23,NaN,Sao Tome and Principe,2020-04-24 03:31:35,4.0,0.0,0.0
18148,18149,2020-04-24,NaN,Sao Tome and Principe,2020-04-25 06:31:19,4.0,0.0,0.0


In [ ]:
def plots_by_country(country):
    cdf = africa_covid[(africa_covid["Country"]==country) & (africa_covid["Province/State"].isnull())][africa_covid.columns[4:]].T
    cdf = cdf.rename(columns={cdf.columns[0]:"confirmed"})
    cfig = px.line(cdf, x=cdf.index, y="confirmed")
    
    delta = (cdf.shift(-1) - cdf).rename(columns={"confirmed": "confirmed per day"})
    cdfig = px.line(delta, x=cdf.index, y="confirmed per day")
    
    delta_p = ((cdf.shift(-1) - cdf) / cdf.shift(-1)).rename(columns={"confirmed": "confirmed per day %"})
    cdpfig = px.line(delta_p, x=cdf.index, y="confirmed per day %")
    return (cfig, cdfig, cdpfig)


In [12]:
af_cov = africa_covid.groupby('Country')['Confirmed', 'Deaths', 'Recovered'].max().reset_index().sort_values('Confirmed', ascending=False)
af_cov

,Country,Confirmed,Deaths,Recovered
37,South Africa,4361.0,86.0,1473.0
11,Egypt,4319.0,307.0,1114.0
26,Morocco,3897.0,159.0,537.0
0,Algeria,3256.0,419.0,1479.0
6,Cameroon,1518.0,53.0,697.0
18,Ghana,1279.0,10.0,134.0
30,Nigeria,1182.0,35.0,222.0
9,Cote d'Ivoire,1077.0,14.0,419.0
10,Djibouti,1008.0,2.0,373.0
42,Tunisia,939.0,38.0,207.0


In [13]:
top_10 = af_cov[:10]
top_10

,Country,Confirmed,Deaths,Recovered
37,South Africa,4361.0,86.0,1473.0
11,Egypt,4319.0,307.0,1114.0
26,Morocco,3897.0,159.0,537.0
0,Algeria,3256.0,419.0,1479.0
6,Cameroon,1518.0,53.0,697.0
18,Ghana,1279.0,10.0,134.0
30,Nigeria,1182.0,35.0,222.0
9,Cote d'Ivoire,1077.0,14.0,419.0
10,Djibouti,1008.0,2.0,373.0
42,Tunisia,939.0,38.0,207.0


In [14]:
top_10 = top_10.set_index('Country')
top_10

,Confirmed,Deaths,Recovered
Country,,,
South Africa,4361.0,86.0,1473.0
Egypt,4319.0,307.0,1114.0
Morocco,3897.0,159.0,537.0
Algeria,3256.0,419.0,1479.0
Cameroon,1518.0,53.0,697.0
Ghana,1279.0,10.0,134.0
Nigeria,1182.0,35.0,222.0
Cote d'Ivoire,1077.0,14.0,419.0
Djibouti,1008.0,2.0,373.0


In [31]:
import plotly.express as px
import plotly.graph_objects as go

for country in top_10.index:
    c_data = top_10.loc[country]
    print('Printing data for ' + country)
    print(c_data)

Printing data for South Africa
Confirmed    4361.0
Deaths         86.0
Recovered    1473.0
Name: South Africa, dtype: float64
Printing data for Egypt
Confirmed    4319.0
Deaths        307.0
Recovered    1114.0
Name: Egypt, dtype: float64
Printing data for Morocco
Confirmed    3897.0
Deaths        159.0
Recovered     537.0
Name: Morocco, dtype: float64
Printing data for Algeria
Confirmed    3256.0
Deaths        419.0
Recovered    1479.0
Name: Algeria, dtype: float64
Printing data for Cameroon
Confirmed    1518.0
Deaths         53.0
Recovered     697.0
Name: Cameroon, dtype: float64
Printing data for Ghana
Confirmed    1279.0
Deaths         10.0
Recovered     134.0
Name: Ghana, dtype: float64
Printing data for Nigeria
Confirmed    1182.0
Deaths         35.0
Recovered     222.0
Name: Nigeria, dtype: float64
Printing data for Cote d'Ivoire
Confirmed    1077.0
Deaths         14.0
Recovered     419.0
Name: Cote d'Ivoire, dtype: float64
Printing data for Djibouti
Confirmed    1008.0
Deaths   